In [29]:
import requests
import pprint
import json
import mysql.connector

state_code = "FL"


url_all_hospitals = f"http://www.communitybenefitinsight.org/api/get_hospitals.php?"
url_hospitals_per_state = f"http://www.communitybenefitinsight.org/api/get_hospitals.php?state={state_code}"

payload = {}
headers = {
    'Cookie': 'PHPSESSID=nb9s8neh28dr9uuol0eg517l2m'
}


#hospitals for florida
response_hospitals_per_state  = requests.request("GET", url_hospitals_per_state, headers=headers, data=payload)

#all Hospitals
response_all_hospitals = requests.request("GET", url_all_hospitals, headers=headers, data=payload)

# Save the JSON object to a file
with open(f"hospital_data_per_state_{state_code}.json", "w") as f:
    json.dump(json.loads(response_hospitals_per_state.content), f, indent=4)

p = pprint.PrettyPrinter(indent=4)
data_all_hosp = json.loads(response_all_hospitals)
data_state_hosp = json.loads(response_hospitals_per_state)
#p.pprint(json.loads(response_hospitals_per_state.content))

Get hospitals per ID

In [27]:
hospital_id = "2164"

url_hospital_data_per_id = f"http://www.communitybenefitinsight.org/api/get_hospital_data.php?hospital_id={hospital_id}"

responce_hospital_data_per_id = requests.request("GET", url_hospital_data_per_id, headers=headers, data=payload)

# Save the JSON object to a file
with open(f"hospital_data_per_id_{hospital_id}.json", "w") as f:
    json.dump(json.loads(responce_hospital_data_per_id.content), f, indent=4)

#p.pprint(json.loads(responce_hospital_data_per_id.content))

In [18]:
#data = json.loads(json_string)
#print((data[0]['name']))

#print all ids where 'name' contains 'state'


#for i in range(len(data)):
    #if  'state' in (data[i]['name']).lower():
        #print(data[i]['hospital_id'])

Shands Jacksonville Medical Center


In [33]:
#connect to mysql

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Konnag78!!',
    database='mydb'
)